In [2]:
! pip install streamlit-searchbox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.3/662.3 kB 5.6 MB/s eta 0:00:0000:0100:01


## Download Data
API Details: https://github.com/binance/binance-public-data/#trades-1

https://data.binance.vision/?prefix=data/spot/daily/trades/BTCUSDT/

In [ ]:
# import wget
import os
import wget
import pandas as pd
from datetime import datetime, timedelta

curr_day = datetime(2021, 3, 1)
# y = "2021"
# m = "03"
# d = "01"
period = "1m"
currency = "BTCUSDT"
data_dir = "./binance_data"

# if not os.path.exists(f"./binance_data/{currency}-trades-{y}-{m}-{d}.csv") and not os.path.exists(f"./binance_data/{currency}-trades-{y}-{m}-{d}.zip"):
#     ! wget -P {data_dir}/ https://data.binance.vision/data/spot/daily/trades/{currency}/{currency}-trades-{y}-{m}-{d}.zip
#     ! unzip ./binance_data/{currency}-trades-{y}-{m}-{d}.zip -d ./binance_data
while (datetime.now()  - curr_day).days > 0:
    
    y, m, d = str(curr_day.year), str(curr_day.month).zfill(2), str(curr_day.day).zfill(2)
    if not os.path.exists(f"./binance_data/{currency}-{period}-{y}-{m}-{d}.csv"):
        ! wget -P {data_dir}/ https://data.binance.vision/data/spot/daily/klines/{currency}/{period}/{currency}-{period}-{y}-{m}-{d}.zip
        ! unzip ./binance_data/{currency}-{period}-{y}-{m}-{d}.zip -d ./binance_data
        ! rm ./binance_data/{currency}-{period}-{y}-{m}-{d}.zip

    curr_day += timedelta(days=1)

## Read Data

In [ ]:
from datetime import datetime, timedelta
import pandas as pd

curr_day = datetime(2021, 3, 1)

currency = "BTCUSDT"
data_dir = "./binance_data"
period = "1m"

combined_df = pd.DataFrame(columns=['OpenTime', 'Open', 'High', 'Low', 'Close', 'Volume', 'CloseTime', 'QuoteAssetVolume', 'NumberOfTrades', 'TakerBuyBaseAssetVolume', 'TakerBuyQuoteAssetVolume', 'Ignore'])

while (datetime.now()  - curr_day).days > 0:

    y, m, d = str(curr_day.year), str(curr_day.month).zfill(2), str(curr_day.day).zfill(2)
    df = pd.read_csv(f"{data_dir}/{currency}-{period}-{y}-{m}-{d}.csv")
    df.loc[-1] = df.columns.values
    df.sort_index(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.columns = ['OpenTime', 'Open', 'High', 'Low', 'Close', 'Volume', 'CloseTime', 'QuoteAssetVolume', 'NumberOfTrades', 'TakerBuyBaseAssetVolume', 'TakerBuyQuoteAssetVolume', 'Ignore']
    df['OpenTimeReadable'] = pd.to_datetime(df['OpenTime'], unit='ms')
    
    combined_df = pd.concat([combined_df, df], ignore_index=True)
    print(f"Done with {y}-{m}-{d}")
    curr_day += timedelta(days=1)

combined_df.to_csv(f"{data_dir}/{currency}-{period}-combined.csv", index=False)
combined_df